# Hooks

使用 Agent 生命周期钩子来监控和响应Agent的各种事件

主要包括两种钩子类型（AgentHooks和RunHooks）、五种核心事件（开始、结束、移交、工具使用开始和结束）

形象理解两种钩子
- AgentHooks：把 agent 视为一个厨师，agenthooks 就类似于厨师的助手，只记录这位厨师的相关事件
- RunHooks：类似于餐馆老板，同一监控和管理每一位厨师的各种事件

区别：AgentHooks 与 agent 绑定，只关注这个 agent 的事件。而 RunHooks 记录所有 agent 的事件

对于Agent钩子，事件有

on_start，该Agent开始时
on_end，该Agent结束时
on_handoff，发生移交时
on_tool_start，工具使用开始时
on_tool_end，工具使用结束时
对于Run钩子，事件有

on_agent_start， 某个Agent开始时
on_agent_end， 某个Agent结束时
on_handoff，发生移交时
on_tool_start，工具使用开始时
on_tool_end，工具使用结束时

下面分别测试 AgentHooks 和 RunHooks 中五类事件的处理

## 开始结束

一般定义钩子类的代码如下：


AgentHooks 作用于单个的 agent，所以在定义 agent 时就传入参数

RunHooks 作用于整个会话中的 agent，所以挂载在 Runner.run() 方法上

```python
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        # 处理开始事件
        pass

    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        # 处理结束事件
        pass

agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, input="instruction ...")
```



```python
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        # 处理开始事件
        pass

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        # 处理结束事件
        pass

agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")

async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="instruction ...")

```

### AgentHooks

In [ ]:
from agents import Agent, Runner, AgentHooks, RunContextWrapper, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# 定义钩子类，继承自AgentHooks
# 这个钩子会在Agent的生命周期中被自动调用
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0  # 事件计数器，用于跟踪事件发生顺序

    # 当Agent开始执行时会自动调用此方法
    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    # 当Agent完成执行时会自动调用此方法
    # TODO output 是默认就是的变量吗？便于在 Runner.run() 运行结束前就输出？
    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended with output {output}"
        )

# 创建Agent实例，绑定钩子
agent = Agent(name="旅行助手", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")

# 运行Agent处理用户问题
result = await Runner.run(agent, "孔子全名叫什么")
# print(result.final_output)  # 打印最终结果

执行流程图

```
Agent.run() 开始
    ↓
await on_start() 钩子  ← 此时无输出
    ↓
Agent 内部处理（LLM推理）
    ↓
生成输出内容
    ↓
await on_end() 钩子    ← 此时有完整输出
    ↓
返回最终结果
```

### RunHooks

使用 RunHooks 来实现，需要更改函数名

In [ ]:
from agents import Agent, Runner, RunHooks, RunContextWrapper, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# 定义钩子，这里只定义了on_start和on_end这两个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended with output {output}"
        )

agent = Agent(name="旅行助手", model=llm, instructions="You are a helpful assistant")
async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="孟子全名叫什么?")
    print(result.final_output)

# if __name__ == "__main__":
#     asyncio.run(main())

await main()

## 使用工具

AgentHooks 和 RunHooks 的工具使用函数名相同，唯一区别在于 `on_tool_end()` 里面多一个 `result`

```python
on_tool_start(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
) -> None

on_tool_end(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
    result: str,
) -> None
```

### AgentHooks

In [ ]:
from agents import Agent, AgentHooks, RunContextWrapper, function_tool, Runner, Tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

# 定义钩子，这里只定义了on_tool_start和on_tool_end这两个事件
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather],  # agent 自己根据 prompt 推理出 city='上海'
    hooks=MyAgentHooks(),
)

async def main():
    result = await Runner.run(agent, "上海最近适合出去玩吗?")
    print(result.final_output)

await main()

# if __name__ == "__main__":
#     asyncio.run(main())

### RunHooks

In [ ]:
from agents import Agent, RunHooks, RunContextWrapper, function_tool, Runner, Tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

# 定义钩子，这里只定义了on_tool_start和on_tool_end这两个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather]  # agent 自己根据 prompt 推理出 city='上海'
)

async def main():
    result = await Runner.run(agent, hooks=MyRunHooks(), input="上海最近适合出去玩吗?")
    print(result.final_output)

await main()

# if __name__ == "__main__":
#     asyncio.run(main())

## 移交

区别：
| 含义 | AgentHooks参数 | RunHooks参数 |
|--|--|--|
| 接收移交的agent  | agent | to_agent |
| 发起移交的agent | source | from_agent |
| 理解：| 从agent的角度来看，自己一般是接收移交的一方 | 从全局角度来看，一般只知道从谁到谁移交  |

```python
class MyAgentHooks(AgentHooks):
    def __init__(self):
        pass
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        pass
```



```python
class MyRunHooks(RunHooks):
    def __init__(self):
        pass
    async def on_handoff(self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent) -> None:
        pass
```

### AgentHooks

In [ ]:
from agents import Agent, AgentHooks, RunContextWrapper, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# 定义钩子，这里只定义了on_handoff这个事件
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0
    
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

history_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
    hooks=MyAgentHooks(),
)

math_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
    hooks=MyAgentHooks(),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_agent, math_agent],  # 移交
    model=llm,
    hooks=MyAgentHooks(),
)

async def main():
    result = await Runner.run(triage_agent, "如何证明勾股定理？")
    print(result.final_output)

    result = await Runner.run(triage_agent, "介绍一下孔子生活的年代?")
    print(result.final_output)

await main()

### RunHooks

In [ ]:
from agents import Agent, RunHooks, RunContextWrapper, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


# 定义钩子，这里只定义了on_handoff这个事件
class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_handoff(self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Handoff from {from_agent.name} to {to_agent.name}."
        )

history_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_agent, math_agent],  # 移交
    model=llm,
)

async def main():
    result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="如何证明勾股定理？")
    print(result.final_output)

    result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="介绍一下孔子生活的年代?")
    print(result.final_output)

await main()